In [1]:
# !wget https://huggingface.co/datasets/mesolitica/put-comma-true-case-audiobook/raw/main/true-case-pasentran-turki.json

In [2]:
import parselmouth
import librosa
import pyworld as pw
from pydub import AudioSegment
from sklearn.preprocessing import StandardScaler
import numpy as np
import os
import malaya_speech
from malaya_speech import Pipeline
import json

`pyaudio` is not available, `malaya_speech.streaming.pyaudio` is not able to use.


In [3]:
import yaml

with open('config.yaml') as fopen:
    config = yaml.safe_load(fopen)
    
config

{'sampling_rate': 22050,
 'fft_size': 1024,
 'hop_size': 256,
 'win_length': None,
 'window': 'hann',
 'num_mels': 80,
 'fmin': 80,
 'fmax': 7600,
 'global_gain_scale': 1.0,
 'trim_silence': True,
 'trim_threshold_in_db': 60,
 'trim_frame_size': 2048,
 'trim_hop_size': 512}

In [4]:
import numpy as np

# https://github.com/TensorSpeech/TensorFlowTTS/blob/master/tensorflow_tts/utils/outliers.py
def is_outlier(x, p25, p75):
    """Check if value is an outlier."""
    lower = p25 - 1.5 * (p75 - p25)
    upper = p75 + 1.5 * (p75 - p25)
    return x <= lower or x >= upper


def remove_outlier(x, p_bottom: int = 25, p_top: int = 75):
    """Remove outlier from x."""
    p_bottom = np.percentile(x, p_bottom)
    p_top = np.percentile(x, p_top)

    indices_of_outliers = []
    for ind, value in enumerate(x):
        if is_outlier(value, p_bottom, p_top):
            indices_of_outliers.append(ind)

    x[indices_of_outliers] = 0.0
    x[indices_of_outliers] = np.max(x)
    return x

In [5]:
import re

_pad = 'pad'
_start = 'start'
_eos = 'eos'
_punctuation = "!'(),.:;? "
_special = '-'
_letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz'

MALAYA_SPEECH_SYMBOLS = (
    [_pad, _start, _eos] + list(_special) + list(_punctuation) + list(_letters)
)

In [6]:
def tts_encode(string: str, add_eos: bool = True):
    r = [MALAYA_SPEECH_SYMBOLS.index(c) for c in string if c in MALAYA_SPEECH_SYMBOLS]
    if add_eos:
        r = r + [MALAYA_SPEECH_SYMBOLS.index('eos')]
    return r

In [7]:
from unidecode import unidecode
import malaya

normalizer = malaya.normalize.normalizer(date = False, time = False, money = False)

def put_spacing_num(string):
    string = re.sub('[A-Za-z]+', lambda ele: ' ' + ele[0] + ' ', string)
    return re.sub(r'[ ]+', ' ', string).strip()

def convert_to_ascii(string):
    return unidecode(string)

def collapse_whitespace(string):
    return re.sub(_whitespace_re, ' ', string)

def cleaning(string, normalize = True, add_eos = False):
    sequence = []
    string = convert_to_ascii(string)
    string = string.replace('&', ' dan ')
    string = re.sub(r'[ ]+', ' ', string).strip()
    string = string
    string = put_spacing_num(string)
    string = ''.join([c for c in string if c in MALAYA_SPEECH_SYMBOLS])
    string = re.sub(r'[ ]+', ' ', string).strip()
    return string, tts_encode(string, add_eos = add_eos)

In [8]:
with open('true-case-pasentran-turki.json') as fopen:
    data = json.load(fopen)

In [9]:
txts = []
directory = 'male-audio'
for k, v in data.items():
    try:
        wav = os.path.join('/home/husein/ssd2', k)
        if os.path.exists(wav):
            txts.append((wav, v['true_case'], directory))
    except Exception as e:
        print(e, k, v)

In [10]:
import IPython.display as ipd
ipd.Audio(txts[1][0])

In [11]:
# cleaning(txts[1][1])

In [12]:
!rm -rf {directory}

In [13]:
!mkdir {directory}

In [14]:
import malaya_speech
import soundfile as sf
import torchaudio
import torch
from malaya_speech import Pipeline
from datasets import Audio
from tqdm import tqdm

vad = malaya_speech.vad.webrtc()

def process(txts, 
            start_silent_trail = int(0.15 * config['sampling_rate']),
            middle_silent_trail = int(0.2 * config['sampling_rate']),
            end_silent_trail = int(0.2 * config['sampling_rate']),
            process_middle_silent = True,
            maxlen = 25):
    
    reader = Audio(sampling_rate = 22050)
    txts = txts[0]
    audios, text_ids = [], []

    for f in tqdm(txts):
        directory = f[2]
        text = f[1]
        f = f[0]
            
        text = cleaning(text)
        try:
            audio = reader.decode_example(reader.encode_example(f))['array']
        except:
            continue
        # audio = audio[start_silent_trail:]

        if config['trim_silence']:
            y_= malaya_speech.resample(audio, config['sampling_rate'], 16000)
            y_ = malaya_speech.astype.float_to_int(y_)
            frames = list(malaya_speech.generator.frames(audio, 30, config['sampling_rate']))
            frames_ = list(malaya_speech.generator.frames(y_, 30, 16000, append_ending_trail = False))
            frames_webrtc = [(frames[no], vad(frame)) for no, frame in enumerate(frames_)]
            grouped_deep = malaya_speech.group.group_frames(frames_webrtc)
            grouped_deep = malaya_speech.group.group_frames_threshold(grouped_deep, 0.15)
            r = []
            for no, g in enumerate(grouped_deep):
                if g[1]:
                    g = g[0].array
                else:
                    if no == 0:
                        g = g[0].array[-start_silent_trail:]
                    elif no == (len(grouped_deep) - 1):
                        g = g[0].array[:end_silent_trail]
                    else:
                        if process_middle_silent:
                            g = np.concatenate([g[0].array[:middle_silent_trail], g[0].array[-middle_silent_trail:]])
                        else:
                            g = g[0].array
                        
                r.append(g)
            audio = np.concatenate(r)
        
        if (len(audio) / config['sampling_rate']) > maxlen:
            print('skipped, audio too long')
            continue

        audio = np.pad(audio, (0, config["fft_size"]), mode="edge")
        f = os.path.split(f)[1]
        new_f = f'{directory}/{f}'
        
        sf.write(new_f, audio, 22050)
        
#         torchaudio.save(new_f, torch.tensor(audio.astype('float32')).unsqueeze(0), 
#                         22050, format='mp3')
        
        audios.append(new_f)
        text_ids.append(text)
    
    return [[audios, text_ids]]

In [15]:
txts[0]

('/home/husein/ssd2/dari-pasentran-ke-istana/output-wav-dari-pasentran-ke-istana/bulan-madu-singkat-11.mp3-0.wav',
 'Sebelas.',
 'male-audio')

In [16]:
i = 135
r = process((txts[i: i + 10],))[0]

100%|████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:02<00:00,  4.37it/s]


In [17]:
r[0]

['male-audio/bulan-madu-singkat-11.mp3-1120.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1121.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1122.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1123.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1124.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1125.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1126.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1127.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1128.wav',
 'male-audio/bulan-madu-singkat-11.mp3-1129.wav']

In [18]:
# import matplotlib.pyplot as plt


# y_ = reader.decode_example(reader.encode_example(r[0][1]))['array']
# plt.plot(y_)

In [19]:
i = 135
# r = process((txts,))[0]

In [24]:
import mp

audios, text_ids = [], []
batch_size = 5000
for i in range(0, len(txts), batch_size):
    print(i)
    index = min(i + batch_size, len(txts))
    b = txts[i: index]
    results = mp.multiprocessing(b, process, cores = 10, returned = True)
    for result in results:
        audios.extend(result[0])
        text_ids.extend(result[1])

0


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 500/500 [02:01<00:00,  4.13it/s]


5000


 61%|███████████████████████████████████████████████████████████▉                                      | 306/500 [01:04<00:42,  4.57it/s]

skipped, audio too long


...

skipped, audio too long

 69%|███████████████████████████████████████████████████████████████████▊                               | 74/108 [00:21<00:08,  4.14it/s]

 64%|███████████████████████████████████████████████████████████████▏                                   | 69/108 [00:23<00:15,  2.48it/s]

skipped, audio too long


100%|██████████████████████████████████████████████████████████████████████████████████████████████████| 108/108 [00:36<00:00,  2.95it/s]


In [27]:
from tqdm import tqdm

data = []
for i in tqdm(range(len(audios))):
    data.append((os.path.join(os.getcwd(), audios[i]), text_ids[i][0]))

100%|██████████████████████████████████████████████████████████████████████████████████████████| 11077/11077 [00:00<00:00, 512552.32it/s]


In [28]:
import json

with open('male-vits.json', 'w') as fopen:
    json.dump(data, fopen)

In [29]:
len(data)

11077